In [17]:
%load_ext autoreload
%autoreload 2

In [1]:
from os.path import join as opj

# data_root = '/om2/user/rogerjin/data'
data_root = '/home/rogerjin/Dropbox/Research/Kellis/masters/code/data'
data_path = opj(data_root, 'data_files_new.npz')

In [2]:
import numpy as np

data = np.load(data_path)
data_keys = list(data.keys())
data_keys

['peak_selector',
 'atac_train_small',
 'atac_test_small',
 'rna_train',
 'rna_test',
 'rna_good_feats',
 'train_idx',
 'test_idx',
 'cell_type_train',
 'cell_type_test']

In [3]:
data['rna_train'].shape

(6897, 7445)

In [4]:
data['rna_test'].shape

(2300, 7445)

In [5]:
data['atac_train_small'].shape

(6897, 3808)

In [6]:
rna_train = data['rna_train']

In [7]:
rna_train[0]

array([-0.16375966, -0.16154247, -0.29548194, ..., -0.19634512,
       -0.4072624 , -0.15920115])

In [8]:
import numpy as np
print(np.sum(rna_train < 0))
np.sum(rna_train > 0)

46208208


5139957

In [12]:
import sys

sys.path.append('/om2/user/rogerjin/GANOLI/ganoli')
sys.path.append('/home/rogerjin/Dropbox/Research/Kellis/masters/code/GANOLI/ganoli')

In [10]:
import torch
torch.cuda.is_available()

False

In [29]:
from GanoliDataset import GanoliUnimodalDataset, GanoliMultimodalDataset

rna = GanoliUnimodalDataset(data['rna_train'])
atac = GanoliUnimodalDataset(data['atac_train_small'])
rna_atac = GanoliMultimodalDataset(rna=rna, atac=atac)

In [31]:
from torch.utils.data import DataLoader

loader = DataLoader(rna_atac, batch_size=5)

for x in loader:
    print(x)
    break

{'rna': tensor([[-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615,  1.9634,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963,  2.0423, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592]],
       dtype=torch.float64), 'atac': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}


In [14]:
for x in loader:
    print(x)
    break

tensor([[-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615,  1.9634,  ..., -0.1963, -0.4073, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963,  2.0423, -0.1592],
        [-0.1638, -0.1615, -0.2955,  ..., -0.1963, -0.4073, -0.1592]],
       dtype=torch.float64)


In [26]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge

model = Ridge(alpha=10**3)
model.fit(data['rna_train'], data['atac_train_small'])

Ridge(alpha=1000)

In [27]:
model.score(data['rna_test'], data['atac_test_small'])

-1.015301847493901